In [1]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from dotenv import load_dotenv
import os
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass
from presentation_generator import Presentation_gen


In [2]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()
presentation=Presentation_gen()

@dataclass
class Deps:

    deep_search_results:list[str]
    quick_search_results:list[str]
    edited_deep_search_result:str
    presentation_slides:dict

agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, what tools to use, always show the output of the tools, except for the presentation agent")

@agent.tool
async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query, generates a report or a paper.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_results.append(res)
    return res

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
@agent.tool
async def quick_research_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_results.append(res.data)
    return res.data
    
editor_agent=Agent(llm, system_prompt="you are an editor, you are given a query, a deep search result, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions")

@agent.tool
async def research_editor_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to edit the deep search result to make it more accurate following the query's instructions.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    res=await editor_agent.run(f'query:{query}, deep_search_result:{ctx.deps.deep_search_results}, quick_search_result:{ctx.deps.quick_search_results}')
    ctx.deps.edited_deep_search_result=res
    return res

@agent.tool
async def presentation_agent(ctx: RunContext[Deps]):
    """
    This function is used to generate a presentation based on the deep search result.
    Args:
        None
    Returns:
        dict: The presentation slides
    """
    if not ctx.deps.edited_deep_search_result:
        res=await presentation.chat(ctx.deps.deep_search_results[0])
    else:   
        res=await presentation.chat(ctx.deps.edited_deep_search_result)
    ctx.deps.presentation_slides=res
    return res


In [3]:
deps=Deps( deep_search_results=[], quick_search_results=[],edited_deep_search_result="")

In [4]:
await agent.run('do a deep research on the topic of the first moon landing',deps=deps)

In [9]:
res

{'page_1': "## Introduction\n\nThe Apollo 11 mission, culminating in the first human landing on the Moon, stands as a monumental achievement in human history. Launched on July 16, 1969, it achieved President Kennedy's goal of landing a man on the moon before the end of the decade.\n\nThe primary objective of Apollo 11 was to safely land humans on the Moon and return them to Earth (Havefunwithhistory.com). This encompassed several detailed objectives, including performing a manned lunar landing (NASA.gov).\n\n![Apollo 11 Launch](https://www.nasa.gov/wp-content/uploads/2023/03/apollo_11_launch_-_gpn-2000-000630_1.jpg)\n\n_Apollo 11 Launch_",
 'page_2': '## Mission Objectives\n\nAccording to NASA.gov, the Apollo 11 mission objectives included:\n\n*   Performing a manned lunar landing\n*   Conducting limited selenological inspection\n*   Capturing photographic documentation of the lunar surface\n*   Performing a survey of the landing site\n\n---\n\n![Lunar Module on the surface](https://mo